In [30]:
import pandas as pd
import numpy as np
from string_grouper import match_strings, match_most_similar, group_similar_strings, StringGrouper
import time

#INPUT_PATH = '~/Desktop/software_analysis_projects/entity_resolution/data/external/csv_example_messy_input.csv'
INPUT_PATH = '~/Desktop/software_analysis_projects/entity_resolution/data/external/csv_example_input_with_true_ids.csv'
OUTPUT_PATH = '~/Desktop/software_analysis_projects/entity_resolution/data/raw/raw_csv_example_messy_input.csv'

In [31]:
df = pd.read_csv(INPUT_PATH)
print(df)
df.info()

         True Id    Id                                 Source  \
0             27  1958           DFSS_AgencySiteLies_2012.csv   
1            221   287  CPS_Early_Childhood_Portal_scrape.csv   
2            224   628  CPS_Early_Childhood_Portal_scrape.csv   
3            316    98  CPS_Early_Childhood_Portal_scrape.csv   
4            316   690  CPS_Early_Childhood_Portal_scrape.csv   
...          ...   ...                                    ...   
3332  4892761244  1339      IDHS_child_care_provider_list.csv   
3333  7237260619   837  CPS_Early_Childhood_Portal_scrape.csv   
3334  7237260619  1758  chapin_dfss_providers_2011_070212.csv   
3335  7237260619  2206           DFSS_AgencySiteLies_2012.csv   
3336  9208960619  2885                NAEYC_accreditation.csv   

                                              Site name  \
0     Chicago Commons\nAssociation St Catherine's - ...   
1           Precious Little One's Learning Center, Inc.   
2                                         

In [27]:
list(df.columns)

['Id',
 'Source',
 'Site name',
 'Address',
 'Zip',
 'Phone',
 'Fax',
 'Program Name',
 'Length of Day',
 'IDHS Provider ID',
 'Agency',
 'Neighborhood',
 'Funded Enrollment',
 'Program Option',
 'Number per Site EHS',
 'Number per Site HS',
 'Director',
 'Head Start Fund',
 'Eearly Head Start Fund',
 'CC fund',
 'Progmod',
 'Website',
 'Executive Director',
 'Center Director',
 'ECE Available Programs',
 'NAEYC Valid Until',
 'NAEYC Program Id',
 'Email Address',
 'Ounce of Prevention Description',
 'Purple binder service type',
 'Column',
 'Column2',
 'name_address_phone']

In [34]:
df['True Id'].value_counts() # count distinct values in single column
#df['True Id'].nunique() # how many unique values in a column

344960624    18
181460608    14
270060608    14
370160623    14
292960623    14
             ..
36060601      1
4060602       1
548060615     1
232361104     1
352060647     1
Name: True Id, Length: 1162, dtype: int64

In [22]:
#df = df.dropna(subset=['Phone'])
df = df[pd.notnull(df['Phone'])]

In [15]:
df['Phone'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
3332    False
3333    False
3334    False
3335    False
3336    False
Name: Phone, Length: 3337, dtype: bool

In [23]:
df['Phone'] = df['Phone'].astype(int)
df['Phone'] = df['Phone'].astype(str)

In [24]:
df['name_address_phone'] = df['Site name'] + ' ' + df['Address'] + ' ' + df['Phone']


In [28]:
print(df)

        Id                                 Source  \
0        0  CPS_Early_Childhood_Portal_scrape.csv   
1        1  CPS_Early_Childhood_Portal_scrape.csv   
2        2  CPS_Early_Childhood_Portal_scrape.csv   
3        3  CPS_Early_Childhood_Portal_scrape.csv   
4        4  CPS_Early_Childhood_Portal_scrape.csv   
...    ...                                    ...   
3332  3332      purple_binder_early_childhood.csv   
3333  3333      purple_binder_early_childhood.csv   
3334  3334      purple_binder_early_childhood.csv   
3335  3335      purple_binder_early_childhood.csv   
3336  3336      purple_binder_early_childhood.csv   

                                              Site name  \
0              Salvation Army - Temple / Salvation Army   
1              Salvation Army - Temple / Salvation Army   
2      National Louis University - Dr. Effie O. Elli...   
3      National Louis University - Dr. Effie O. Elli...   
4      Board Trustees-City Colleges of Chicago - Oli...   
...      

In [29]:
df[df.duplicated(['name_address_phone'])]

,Id,Source,Site name,Address,Zip,Phone,Fax,Program Name,Length of Day,IDHS Provider ID,...,Center Director,ECE Available Programs,NAEYC Valid Until,NAEYC Program Id,Email Address,Ounce of Prevention Description,Purple binder service type,Column,Column2,name_address_phone
1,1,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Salvation Army - Temple / Salvation Army 1 N ...
3,3,CPS_Early_Childhood_Portal_scrape.csv,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave,NaN,5339011,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,National Louis University - Dr. Effie O. Elli...
5,5,CPS_Early_Childhood_Portal_scrape.csv,Board Trustees-City Colleges of Chicago - Oli...,10001 S Woodlawn Ave,NaN,2916100,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Board Trustees-City Colleges of Chicago - Oli...
7,7,CPS_Early_Childhood_Portal_scrape.csv,Easter Seals Society of Metropolitan Chicago ...,1001 W Roosevelt Rd,NaN,9395115,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Easter Seals Society of Metropolitan Chicago ...
10,10,CPS_Early_Childhood_Portal_scrape.csv,Hull House Association - Child Dev. Central O...,1030 W Van Buren St,NaN,9068600,NaN,Child Care,EXTENDED DAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hull House Association - Child Dev. Central O...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,3316,purple_binder_early_childhood.csv,El Hogar Del Nino,1718 S Loomis Street,60608.0,5639796,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early Head Start,El Hogar Del Nino 1718 S Loomis Street 5639796
3317,3317,purple_binder_early_childhood.csv,Erie Neighborhood House,1347 W Erie Street,60642.0,6663430,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early Head Start,Erie Neighborhood House 1347 W Erie Street 6...
3323,3323,purple_binder_early_childhood.csv,Marcy Newberry Association,1073 W Maxwell Street,60608.0,8297555,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early Head Start,Marcy Newberry Association 1073 W Maxwell Str...
3325,3325,purple_binder_early_childhood.csv,North Avenue Day Nursery,2001 W Pierce Street,60622.0,3424499,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early Head Start,North Avenue Day Nursery 2001 W Pierce Street...


In [26]:
start_time = time.time()
# Create all matches:
matches = match_strings(df['name_address_phone'])
# Look at only the non-exact matches:
matches[matches.left_side != matches.right_side].head()

print(matches)
print("--- %s seconds ---" % (time.time() - start_time))
matches.to_csv(OUTPUT_PATH)
print(matches.info())

                                              left_side  \
0      Salvation Army - Temple / Salvation Army 1 N ...   
1      Salvation Army - Temple / Salvation Army 1 N ...   
2      Salvation Army - Temple / Salvation Army 1 N ...   
3      Salvation Army - Temple / Salvation Army 1 N ...   
4      Salvation Army - Temple / Salvation Army 1 N ...   
...                                                 ...   
9180   YMCA North Lawndale 3449 W Arthington Street ...   
9181   YMCA North Lawndale 3449 W Arthington Street ...   
9182   C.C.C. Learning Center 219 S Dearborn Street ...   
9183   Haymarket Center 932 W. Washington Blvd  2267984   
9184   Haymarket Center 932 W. Washington Blvd  2267984   

                                             right_side  similarity  
0      Salvation Army - Temple / Salvation Army 1 N ...    1.000000  
1      Salvation Army - Temple / Salvation Army 1 N ...    1.000000  
2      Salvation Army - Temple / Salvation Army 1 N ...    1.000000  
3      Salv

In [2]:
df = pd.read_csv(OUTPUT_PATH)


In [3]:
df.sort_values(by='similarity')

,Unnamed: 0,left_side,right_side,similarity
8249,8249,Abraham Lincoln Center Lincoln/King,Abraham Lincoln Center Abraham-Lincoln,0.800101
8234,8234,Abraham Lincoln Center Abraham-Lincoln,Abraham Lincoln Center Lincoln/King,0.800101
1258,1258,Abraham Lincoln Centre - Lincoln - Roseland,Abraham Lincoln Centre,0.800114
1275,1275,Abraham Lincoln Centre - Lincoln - Roseland,Abraham Lincoln Centre,0.800114
1274,1274,Abraham Lincoln Centre - Lincoln - Roseland,Abraham Lincoln Centre,0.800114
...,...,...,...,...
9479,9479,Mary Crane League Mary Crane Center (St. Marti...,Mary Crane League Mary Crane Center (St. Marti...,1.000000
2732,2732,Northwestern University Settlement House - No...,Northwestern University Settlement House - No...,1.000000
2731,2731,Northwestern University Settlement House - No...,NORTHWESTERN UNIVERSITY SETTLEMENT HOUSE NORTH...,1.000000
9512,9512,Northwestern University Settlement House NORTH...,Northwestern University Settlement House - No...,1.000000


In [5]:
df.tail()


,Unnamed: 0,left_side,right_side,similarity
12797,12797,C.C.C. Learning Center,C.C.C. Learning Center,1.000000
12798,12798,Haymarket Center,Haymarket Center,1.000000
12799,12799,Haymarket Center,Haymarket Center Haymarket Center,0.974341
12800,12800,Haymarket Center,HAYMARKET CENTER HAYMARKET CENTER,0.974341
12801,12801,Haymarket Center,Haymarket Center - Haymarket Center,0.974341


In [6]:
df

,Unnamed: 0,left_side,right_side,similarity
0,0,Salvation Army - Temple / Salvation Army,Salvation Army - Temple / Salvation Army,1.000000
1,1,Salvation Army - Temple / Salvation Army,Salvation Army - Temple / Salvation Army,1.000000
2,2,Salvation Army - Temple / Salvation Army,Salvation Army - Temple / Salvation Army,1.000000
3,3,Salvation Army - Temple / Salvation Army,Salvation Army - Temple / Salvation Army,1.000000
4,4,Salvation Army - Temple / Salvation Army,Salvation Army - Temple / Salvation Army,1.000000
...,...,...,...,...
12797,12797,C.C.C. Learning Center,C.C.C. Learning Center,1.000000
12798,12798,Haymarket Center,Haymarket Center,1.000000
12799,12799,Haymarket Center,Haymarket Center Haymarket Center,0.974341
12800,12800,Haymarket Center,HAYMARKET CENTER HAYMARKET CENTER,0.974341
